# 02 - Guía profunda de DateTime, Timestamp y Timedelta

Objetivos avanzados:
- Entender `pd.Timestamp` (naive vs aware), `datetime` de Python y `pd.Timedelta`.
- Parseo robusto con `pd.to_datetime` (formatos mixtos, epoch, `errors`, `dayfirst`, `utc`).
- Zonas horarias: `tz_localize` vs `tz_convert`, casos con DST (horas inexistentes/ambiguas).
- Índices temporales: `DatetimeIndex`, `date_range`, slicing, `resample` vs `asfreq`, `rolling`.
- `merge_asof` para unir por cercanía temporal y alineación de granularidades.


## Tipos base y terminología

- `datetime.datetime` (Python): objeto estándar; puede ser naive (sin tz) o aware (con tz).
- `pd.Timestamp`: envoltura mejorada para trabajar con pandas; recomienda usarse en series/índices.
- `pd.Timedelta`: diferencia de tiempo (duración); permite aritmética (`Timestamp` ± `Timedelta`).
- `NaT`: valor faltante temporal (análogo a `NaN`).
- Naive vs aware:
  - Naive: no tiene zona horaria ⇒ ambiguo globalmente.
  - Aware: tiene tz ⇒ representa un instante absoluto; preferible trabajar en UTC internamente.


In [1]:
import pandas as pd
from datetime import datetime, timezone

print("Creación de Timestamps y Timedeltas; naive vs aware:")
naive = pd.Timestamp("2024-01-01 10:00:00")
aware_utc = pd.Timestamp("2024-01-01 16:00:00", tz="UTC")
aware_mx = pd.Timestamp("2024-01-01 10:00:00", tz="America/Mexico_City")

print("naive:", naive, "| tz=", naive.tz)
print("aware_utc:", aware_utc, "| tz=", aware_utc.tz)
print("aware_mx:", aware_mx, "| tz=", aware_mx.tz)

print("\nAritmética con Timedelta:")
una_hora = pd.Timedelta(hours=1, minutes=30)
print("una_hora:", una_hora)
print("naive + 1h30m =>", naive + una_hora)
print("aware_utc - 1h30m =>", aware_utc - una_hora)

print("\nComponentes y normalización:")
print("año/mes/día:", naive.year, naive.month, naive.day)
print("floor('D') (trunca a día):", naive.floor('D'))
print("ceil('H') (redondea arriba a hora):", naive.ceil('H'))
print("round('H') (redondeo estándar):", pd.Timestamp('2024-01-01 10:31').round('H'))

print("\nConversión de tz: Mexico City -> UTC:")
print("aware_mx.tz_convert('UTC') =>", aware_mx.tz_convert("UTC"))


Creación de Timestamps y Timedeltas; naive vs aware:
naive: 2024-01-01 10:00:00 | tz= None
aware_utc: 2024-01-01 16:00:00+00:00 | tz= UTC
aware_mx: 2024-01-01 10:00:00-06:00 | tz= America/Mexico_City

Aritmética con Timedelta:
una_hora: 0 days 01:30:00
naive + 1h30m => 2024-01-01 11:30:00
aware_utc - 1h30m => 2024-01-01 14:30:00+00:00

Componentes y normalización:
año/mes/día: 2024 1 1
floor('D') (trunca a día): 2024-01-01 00:00:00
ceil('H') (redondea arriba a hora): 2024-01-01 10:00:00
round('H') (redondeo estándar): 2024-01-01 11:00:00

Conversión de tz: Mexico City -> UTC:
aware_mx.tz_convert('UTC') => 2024-01-01 16:00:00+00:00


/tmp/ipykernel_239545/1122924106.py:22: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  print("ceil('H') (redondea arriba a hora):", naive.ceil('H'))
/tmp/ipykernel_239545/1122924106.py:23: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  print("round('H') (redondeo estándar):", pd.Timestamp('2024-01-01 10:31').round('H'))


## Parseo robusto con `pd.to_datetime`

- Formatos mixtos: pandas intenta inferir; usa `errors='coerce'` para invalidos → `NaT`.
- `utc=True`: entrega `Timestamp` aware en UTC (recomendado para pipelines).
- `dayfirst=True`: interpreta el primer número como día.
- Epoch: usa `unit=` (p. ej. `s`, `ms`, `us`, `ns`).
- Limpieza: estandariza a UTC y luego convierte al presentar.


In [ ]:
import numpy as np

print("Parseo de formatos mixtos y epoch:")
crudos = pd.Series([
    "2024-01-01 08:00:00-06:00",
    "01-02-2024 10:30",     # dd-mm-YYYY (dayfirst)
    1704115200,              # epoch seconds (2024-01-01 UTC)
    "2024/01/04",
    "no-fecha",
])
fechas = pd.to_datetime(crudos, utc=True, errors="coerce", dayfirst=True)
print(fechas)
print("\nNormalizamos a día (floor) para comparar granularidades:")
print(fechas.dt.floor('D'))


## Zonas horarias y DST (horas inexistentes/ambiguas)

- `tz_localize`: asigna tz a timestamps naive (no cambia el instante, define referencia).
- `tz_convert`: cambia de zona manteniendo el instante absoluto.
- Horas inexistentes: cuando el reloj salta hacia adelante (inicio del DST).
- Horas ambiguas: cuando el reloj retrocede (fin del DST).
- Estrategias: `ambiguous='infer'|'NaT'|bool-array`, `nonexistent='shift_forward'|'NaT'|'shift_backward'`.


In [ ]:
print("Localización con casos problemáticos (America/New_York para ilustrar DST):")
naive_amb = pd.to_datetime(["2022-11-06 01:30", "2022-03-13 02:30"])  # ambigua, inexistente
ny = "America/New_York"

try:
    print("Ambigua 2022-11-06 01:30 -> ambiguous='infer':")
    print(naive_amb[0].tz_localize(ny, ambiguous='infer'))
except Exception as e:
    print("Ambigua requiere 'ambiguous' explícito:", e)

try:
    print("Inexistente 2022-03-13 02:30 -> nonexistent='shift_forward':")
    print(naive_amb[1].tz_localize(ny, nonexistent='shift_forward'))
except Exception as e:
    print("Inexistente requiere 'nonexistent' explícito:", e)

print("\nConvertir luego a UTC:")
print(naive_amb[0].tz_localize(ny, ambiguous=True).tz_convert("UTC"))


## Índices temporales, `resample` vs `asfreq`, `rolling`, y filtros por hora

- `date_range`: genera `DatetimeIndex` regulares.
- `resample`: agrega a una frecuencia (sum, mean, count, etc.).
- `asfreq`: cambia frecuencia SIN agregar (introduce `NaN` donde falten observaciones).
- Relleno: `ffill`/`bfill` tras `asfreq`.
- Ventanas móviles: `rolling('30min')` en series irregulares; o por número de observaciones.
- Filtros: `.between_time('09:00','17:00')`, `.at_time('12:00')`.


In [ ]:
print("Serie irregular y operaciones de frecuencia:")
idx = pd.to_datetime([
    "2024-01-01 09:00", "2024-01-01 09:10", "2024-01-01 09:55",
    "2024-01-01 10:20", "2024-01-01 11:00"
])
s = pd.Series([5, 3, 2, 7, 4], index=idx)
print(s)

print("\nresample('H').sum(): agrega por hora (09, 10, 11):")
print(s.resample('H').sum())

print("\nasfreq('15min'): cambia la frecuencia sin agregar (introduce NaN):")
sa = s.asfreq('15min')
print(sa.head(10))

print("\nRelleno forward-fill tras asfreq:")
print(sa.ffill().head(10))

print("\nRolling de 30 minutos (suma) sobre serie irregular:")
print(s.rolling('30min').sum())

print("\nFiltros por hora de oficina (09:00-11:00):")
df = s.to_frame('valor')
print(df.between_time('09:00', '11:00').head())
print("\nFiltra exactamente a las 10:00 (at_time):")
print(df.at_time('10:00'))


## `merge_asof`: combinación por cercanía temporal

- Une observaciones alineando por la clave temporal más cercana (por defecto, hacia atrás).
- Útil para asignar la última cotización conocida a cada transacción, o sensores de distinta frecuencia.
- Parámetros: `by` (clave adicional exacta), `tolerance` (máxima distancia), `direction` (`backward`/`forward`/`nearest`).


In [ ]:
print("Demostración de merge_asof con trades y quotes:")
quotes = pd.DataFrame({
    'time': pd.to_datetime(['2024-01-01 09:00:00','2024-01-01 09:00:30','2024-01-01 09:02:00']),
    'symbol': ['AAA','AAA','AAA'],
    'price': [10.0, 10.1, 10.2]
})
trades = pd.DataFrame({
    'time': pd.to_datetime(['2024-01-01 09:00:10','2024-01-01 09:01:00','2024-01-01 09:02:30']),
    'symbol': ['AAA','AAA','AAA'],
    'size': [100, 50, 80]
})

print("quotes:")
print(quotes)
print("\ntrades:")
print(trades)

joined = pd.merge_asof(
    trades.sort_values('time'),
    quotes.sort_values('time'),
    on='time',
    by='symbol',
    tolerance=pd.Timedelta('1min'),
    direction='backward'
)
print("\nmerge_asof (última cotización previa <= 1min):")
print(joined)


## Periodos vs Timestamps

- `Period` representa intervalos (p. ej., mes 2024-01) y mantiene semántica de período.
- `period_range` para generar `PeriodIndex`.
- Conversión: `.to_timestamp()` (inicio/fin de período) y `.to_period('M')` para agrupar por mes con semántica clara.


In [ ]:
print("PeriodIndex mensual y conversión a Timestamp:")
per = pd.period_range('2024-01', periods=3, freq='M')
print(per)
print("\nA inicio de periodo (to_timestamp):")
print(per.to_timestamp())

print("\nAgrupar por mes con to_period('M') y sumar:")
idx = pd.date_range('2024-01-01', periods=90, freq='D')
vals = pd.Series(range(90), index=idx)
print(vals.to_period('M').groupby(level=0).sum())


# 02 - Timestamps, Zonas Horarias y Frecuencias

Objetivos:
- Normalizar tiempos con `to_datetime` y trabajar en UTC.
- Localizar/convertir zonas horarias y comparar.
- Usar tiempos como índice, `resample` y `date_range`.


In [ ]:
import pandas as pd
import numpy as np

print("Parseo y normalización a UTC:")
crudos = pd.Series([
    "2024-01-01 08:00:00-06:00",
    "2024/01/01 08:30:00 -0600",
    "01-01-2024 09:00",  # naive
    None,
])
fechas = pd.to_datetime(crudos, utc=True, errors="coerce")
print(fechas)

print("\nCrear DataFrame con naive y luego localizar tz:")
df = pd.DataFrame({
    "evento": ["A", "B", "C"],
    "t_local": pd.to_datetime(["2024-01-01 10:00", "2024-01-01 10:05", "2024-01-01 10:10"], errors="coerce")
})
df["t_local"] = df["t_local"].dt.tz_localize("America/Mexico_City")
print(df)


## Localizar vs Convertir tz y comparaciones

- `tz_localize`: asigna zona a naive (no cambia momento real, define referencia).
- `tz_convert`: transforma a otra zona (mismo instante, cambia representación).
- Comparar granularidades: truncar/ajustar con `.dt.floor('D')`, `.dt.date` u otras operaciones.


In [ ]:
print("Convertir a UTC y comparar:")
df["t_utc"] = df["t_local"].dt.tz_convert("UTC")
print(df[["t_local", "t_utc"]])

print("\nComparación por día (floor a 'D'):")
df["dia"] = df["t_utc"].dt.floor("D")
print(df[["evento", "dia"]])


## Índice temporal, slicing y resample

- Indexa por tiempo para habilitar slicing (`df['2024-01-01']`).
- `resample('H')` agrega por hora, `resample('D')` por día, etc.
- `date_range` genera series de tiempo regulares.


In [ ]:
print("Crear serie cada 15 minutos y agregar por hora:")
idx = pd.date_range("2024-01-01 00:00", periods=8, freq="15min", tz="UTC")
s = pd.Series(range(8), index=idx)
print("Suma por hora:")
print(s.resample("H").sum())

print("\nSlicing por fecha:")
print(s["2024-01-01 00:30":"2024-01-01 01:15"])
